In [804]:
# Importing Necessary Libraries
import warnings
warnings.filterwarnings("ignore")

import math
import pandas as pd
import numpy as np
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [805]:
food_data = pd.read_csv("Capital_Area_Food_Bank_Hunger_Estimates.csv")

In [806]:
print(food_data.shape)
print(food_data.columns)
print(food_data[food_data.isnull().any(axis=1)])

(1039, 42)
Index(['OBJECTID', 'STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10',
       'NAMELSAD10', 'MTFCC10', 'FUNCSTAT10', 'ALAND10', 'AWATER10',
       'GEOGRAPH', 'TRACT', 'POVERTY200', 'TRACTCE', 'PERCENT_CA',
       'PERCENT_TR', 'PERCENT_WA', 'TOTAL_POP', 'UNEMPLOYME', 'POVERTY_RA',
       'MEDIAN_INC', 'PERCENT_BL', 'PERCENT_HI', 'HOME_OWN', 'F15_FI_RATE',
       'F15_FI_POP', 'F15_LB_NEED', 'F15_LB_UNME', 'F15_DISTRIB', 'F15_PPIN',
       'FY_FI_RATE', 'FY_FI_POP', 'FY_LB_UNME', 'FY_DISTRIB', 'FY_PPIN',
       'F14_FI_RATE', 'F14_LB_UNME', 'F14_DISTRIB', 'F14_PPIN', 'SHAPEAREA',
       'SHAPELEN'],
      dtype='object')
Empty DataFrame
Columns: [OBJECTID, STATEFP10, COUNTYFP10, TRACTCE10, GEOID10, NAME10, NAMELSAD10, MTFCC10, FUNCSTAT10, ALAND10, AWATER10, GEOGRAPH, TRACT, POVERTY200, TRACTCE, PERCENT_CA, PERCENT_TR, PERCENT_WA, TOTAL_POP, UNEMPLOYME, POVERTY_RA, MEDIAN_INC, PERCENT_BL, PERCENT_HI, HOME_OWN, F15_FI_RATE, F15_FI_POP, F15_LB_NEED, F15_LB_UNME, F15_DI

In [807]:
print('We have {} rows and {} columns.'.format(food_data.shape[0], food_data.shape[1]))
print(food_data.head(2))

We have 1039 rows and 42 columns.
   OBJECTID  STATEFP10  COUNTYFP10  TRACTCE10      GEOID10   NAME10  \
0        26         24          31     703402  24031703402  7034.02   
1        27         24          31     701202  24031701202  7012.02   

             NAMELSAD10 MTFCC10 FUNCSTAT10  ALAND10  ...  FY_FI_POP  \
0  Census Tract 7034.02   G5020          S  1257675  ...   0.000000   
1  Census Tract 7012.02   G5020          S  1841629  ...  87.030944   

     FY_LB_UNME   FY_DISTRIB    FY_PPIN  F14_FI_RATE   F14_LB_UNME  \
0      0.000000     0.000000   0.000000          2.0  12250.272452   
1  14863.288579  3413.209661  39.218346          3.6  17749.107675   

   F14_DISTRIB   F14_PPIN     SHAPEAREA     SHAPELEN  
0      2038.13  29.954845  1.257565e+06  4579.987688  
1      2496.57  25.895904  1.848218e+06  6774.565707  

[2 rows x 42 columns]


The dataset we are using contains the data from the DC area food banks on the food given out at different food banks over several years. There are features such as what was needed by the community and what was sent to the food bank that year. Overall, it consists of 1032 rows and 42 columns throughout. We'll go in depth about each of these individual columns below and analyze whether or not they're going to be useful to our project throughout.

List of Columns: OBJECTID, STATEFP10, COUNTYFP10, TRACTCE10, GEOID10, NAME10, NAMELSAD10, MTFCC10, FUNCSTAT10, ALAND10, AWATER10, GEOGRAPH, TRACT, POVERTY200, TRACTCE, PERCENT_CA, PERCENT_TR, PERCENT_WA, TOTAL_POP, UNEMPLOYME, POVERTY_RA, MEDIAN_INC, PERCENT_BL, PERCENT_HI, HOME_OWN, F15_FI_RATE, F15_FI_POP, F15_LB_NEED, F15_LB_UNME, F15_DISTRIB, F15_PPIN, FY_FI_RATE, FY_FI_POP, FY_LB_UNME, FY_DISTRIB, FY_PPIN, F14_FI_RATE, F14_LB_UNME, F14_DISTRIB, F14_PPIN, SHAPEAREA, SHAPELEN

Sadly, looking at the list of columns in the data, we were never really given a description of what some of these features refer to. I'll separate them in the list below based on what information we are able to go and derive based on the name and, following, information that we're unsure about.

<ul>
    <b>ID Columns</b>
    <li>ObjectID (int64): The unique ID of the row/item being discussed</li>
    <li>StateFP10 (int64): The state ID of the row/item being discussed</li>
    <li>CountyFP10 (int64): The county ID of the row/item being discussed</li>
    <li>TractFP10 (int64): The tract ID of the row/item being discussed</li>
    <li>GeoID10 (int64): The geographic ID of the row/item being discussed</li>
    <li>Name10 (float): The name of the row/item being discussed</li>
    <li>NameLSad10 (object): The longer/more descriptive name of the row/item</li>
</ul>

These columns specifically go and refer to the individual name/ID of the land/tract/etc. being discussed in the row throughout. Each of these values mentioned above act as an identifier that specifically helps to distinguish the item described from those whom might be similar regarding other traits.

<ul>
    <b>Attribute Columns</b>
    <li>ALand10 (int64): Land area of the row/item being discussed</li>
    <li>AWater10 (int64): Water area of the row/item being discussed</li>
    <li>TotalPop (int64): Total population of the row/item being discussed</li>
    <li>Unemployme (float): Unemployment rate of the row/item being discussed</li>
    <li>PovertyRa (float): Poverty rate of the row/item being discussed</li>
    <li>MedianInc (float): Median income of the row/item being discussed</li>
    <li>PercentBl (float): Percentage of African-Americans who live in the row/item being discussed</li>
    <li>PercentHi (float): Percentage of Hispanics who live in the row/item being discussed</li>
    <li>HomeOwn (float): Percentage of homeowners who live in the row/item being discussed</li>
</ul>

These columns delve further into the traits/attributes that help to characterize the land/tract/etc. being discussed in the row throughout. These features in particular will probably be some of those that help the most with feature engineering in particular, as they're the ones that specifically differentiate the pieces of land from each other with regards to demographics, helping us to get further insights into the population and individuals that live there.

<ul>
    <b>Food/Distribution Columns</b>
    <li>Fx_FI_Rate (float): The estimated portion of the population in the census tract experiencing food insecurity</li>
    <li>Fx_FI_Pop (float): The estimated number of people in the census tract experiencing food insecurity</li>
    <li>Fx_LB_Need (float): The estimated pounds of food needed by the food insecure population in the census tract</li>
    <li>Fx_Distrib (float): The number of pounds of food distributed by CAFB and partners in the census tract</li>
    <li>Fx_LB_Unme (float): The difference between the estimated pounds of food needed and the real pounds of food distributed</li>
</ul>

These columns are perhaps the most important, as they specifically have to do with the target variable that we'll select/look into. The column 'Fx_LB_Unme' will perhaps be later selected to be the target, as it in particular deals with the either the amount of unnecessary food delivered or the amount of food lacking throughout.

<ul>
    <b>Unknown Columns</b>
    <li>MTFCC10 (object): Seems like another ID value but not completely sure.</li>
    <li>FuncStat10 (object): Seems like another either ID value or attribute but not completey sure</li>
</ul>

Both of these columns don't seem to be that important at all, and don't offer much information throughout from what I can tell. Unless I find out that they're incredibly valuable as a whole when I look into analysis further, I'm probably going to drop them from consideration regarding any models built.

<ul>
    <b>Unknown Columns</b>
    <li>Percent_CA (object): Seems like an attribute column, could be useful for feature engineering</li>
    <li>Percent_TR (object): Seems like an attribute column, could be useful for feature engineering</li>
    <li>Percent_WA (object): Seems like an attribute column, could be useful for feature engineering</li>
</ul>

These columns are definitely attributes of a given census tract, but there's no indication of what they stand for. Nevertheless, they'll probably come in handy when taking feature engineering into consideration.

In [808]:
num_cols = food_data._get_numeric_data().columns
categorical_columns = list(set(food_data.columns) - set(num_cols))
categorical_columns

['NAMELSAD10', 'FUNCSTAT10', 'MTFCC10']

As mentioned above, I'm just going to remove each of the columns above. We have no idea what 'MTFCC10' and 'FUNCSTAT10' refer to, and it seems like all the information to be gained from 'NAMELSAD10' is contained in the abbreviated 'NAME' column. There's nothing further to be gained from simply encoding each individual value in 'NAMELSAD10', given that there won't be any overlap.

In [809]:
food_data.drop(['FUNCSTAT10', 'NAMELSAD10', 'MTFCC10'], inplace=True, axis=1)

Besides this, we're going to go and drop all those columns that are either referring to ID or just those that we don't specifically know what they mean in regards to their definition. These won't be too helpful in regards to our models throughout.

In [810]:
columns_to_drop = ["F15_LB_UNME", "FY_FI_RATE", "FY_FI_POP", 
                   "FY_LB_UNME", "FY_DISTRIB", "FY_PPIN", "OBJECTID", 
                   "STATEFP10", "COUNTYFP10", "TRACTCE10", "GEOID10", 
                   "NAME10"]

for col in columns_to_drop:
    food_data.drop(col, inplace=True, axis=1)

Next, I'll go and create a simple model so that we can get a baseline score for the dataset throughout, without any feature engineering or advanced models throughout. I don't expect this score to be any good, given that the established columns provided don't seem too predictive as a whole just yet.

In [811]:
train_y = food_data['F15_LB_NEED'].values
train_x = food_data.drop('F15_LB_NEED', axis=1)
xtra, xte, ytra, yte = train_test_split(
    train_x, train_y, test_size=0.2)

rf_params = {}
rf_model = RandomForestRegressor()
rf_model.fit(xtra, ytra)
train_preds = rf_model.predict(xte)

print("Current R2: {}".format(r2_score(yte, train_preds)))
print("Current RMSE: {}".format(math.sqrt(mean_squared_error(yte, train_preds))))

Current R2: 0.9982414591315324
Current RMSE: 3043.799396200841


It looks like I missed some leakage. There's definitely one column that's extremely predictive that is used to go and calculate the correct amount of food for each census tract. Let's go and look into the RandomForest feature importances and see which feature it happens to be.

In [812]:
feature_importances = pd.DataFrame(
    rf_model.feature_importances_, index = train_x.columns,
    columns=['importance']).sort_values('importance', ascending=False)

feature_importances[:1]

,importance
F15_FI_POP,0.991564


As expected, one column happens to be extremely predictive; in this case it happens to be "F15_FI_Pop", which refers to the estimated number of people in the census tract whom happen to be experiencing food insecurity. I expect what happens is that the individuals whom decide how much food is needed simply multiply the total population in need by a scalar value. Realizing this, we now have to go and change up what we're trying to predict/work with, as there's no point in predicting a static value that explicitly is based off another value. However, although we may still change things up, we'll also go and repeat the same methodology as above, except with the "F15_FI_Pop" column removed. This will allow for us to get a feel for the overall predictiveness of the other features.

In [813]:
food_data = food_data.drop(["F15_FI_POP", "F14_LB_UNME"], axis=1)

train_y = food_data['F15_LB_NEED'].values
train_x = food_data.drop('F15_LB_NEED', axis=1)
xtra, xte, ytra, yte = train_test_split(
    train_x, train_y, test_size=0.2)

rf_params = {}
rf_model = RandomForestRegressor()
rf_model.fit(xtra, ytra)
train_preds = rf_model.predict(xte)

print("Current R2: {}".format(r2_score(yte, train_preds)))
print("Current RMSE: {}".format(math.sqrt(mean_squared_error(yte, train_preds))))

Current R2: 0.9795375418220428
Current RMSE: 9530.829425220334


In [814]:
feature_importances = pd.DataFrame(
    rf_model.feature_importances_, index = train_x.columns,
    columns=['importance']).sort_values('importance', ascending=False)

feature_importances[:5]

,importance
F15_FI_RATE,0.451985
F15_DISTRIB,0.211830
TOTAL_POP,0.122675
F14_FI_RATE,0.105980
F14_DISTRIB,0.054189


That's a little more useful, as the overall distribution/rate seems to correlate with the overall need as a whole. We're probably still going to go and modify our target in order to predict a more useful statistic--probably the amount of wasted/unmet food so that CAFB and partners can optimize their overall distribution. Overall, though, we're going to have to go and transform the data, splitting by year so that we can get a sense of how predictive our model truly is. Our new target column will be "LB_UNME". We've also decided that we're going to drop all of the attribute/other columns, given their lack of helpfulness to the overall model throughout when looking at the RandomForest feature importances.

In [815]:
food_data = pd.read_csv("Capital_Area_Food_Bank_Hunger_Estimates.csv")

In [816]:
f15_columns = ['F15_FI_RATE', 'F15_LB_UNME', 'F15_DISTRIB', 'F15_PPIN']
f14_columns = ['F14_FI_RATE', 'F14_LB_UNME', 'F14_DISTRIB', 'F14_PPIN']

f14_initial = [col for col in food_data.columns if col not in f14_columns]
f14_final = [col for col in f14_initial if col not in f15_columns] + f14_columns
f15_initial = [col for col in food_data.columns if col not in f15_columns]
f15_final = [col for col in f15_initial if col not in f14_columns] + f15_columns

In [817]:
food_data_2014 = food_data[f14_final]
food_data_2015 = food_data[f15_final]

f14_rename = {'F14_FI_RATE': "FI_RATE", 'F14_LB_UNME': "LB_UNME", 'F14_DISTRIB': "DISTRIB", 'F14_PPIN': "PPIN"}
f15_rename = {'F15_FI_RATE': "FI_RATE", 'F15_LB_UNME': "LB_UNME", 'F15_DISTRIB': "DISTRIB", 'F15_PPIN': "PPIN"}
cols_to_use = ["LB_UNME", "FI_RATE", "TOTAL_POP", "PPIN", "DISTRIB"]

food_data_2014 = food_data_2014.rename(columns=f14_rename)[cols_to_use]
food_data_2015 = food_data_2015.rename(columns=f15_rename)[cols_to_use]

In [818]:
train_y = food_data_2014['LB_UNME'].values
train_x = food_data_2014.drop('LB_UNME', axis=1)
xtra, xte, ytra, yte = train_test_split(
    train_x, train_y, test_size=0.2, random_state=2019)

rf_params = {}
rf_params["n_estimators"] = 100
rf_params["max_depth"] = 5
rf_params["random_state"] = 2019

rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(xtra, ytra)
train_preds = rf_model.predict(xte)

test_y = food_data_2015["LB_UNME"]
test_x = food_data_2015.drop('LB_UNME', axis=1)
test_preds = rf_model.predict(test_x)

print("Current R2: {}".format(r2_score(yte, train_preds)))
print("Current RMSE: {}".format(math.sqrt(mean_squared_error(yte, train_preds))))
print("Current R2 (2015): {}".format(r2_score(test_y, test_preds)))
print("Current RMSE (2015): {}".format(math.sqrt(mean_squared_error(test_y, test_preds))))

Current R2: 0.9307301324557461
Current RMSE: 11015.170945393511
Current R2 (2015): -1.4735078589514057
Current RMSE (2015): 69461.93916524405


Straight off the bat, we can see that our model is pretty awful at generalizing, given the low R2 score and the high RMSE in comparison. The next order of business is to go and work on improving this value so that we can explicitly determine where resources would be best served. In the following cell, we're going to focus on scaling throughout to see if it ends up being helpful to the model.

In [819]:
scaler = StandardScaler()

train_unmet = food_data_2014["LB_UNME"].values
food_data_2014_scaled = food_data_2014.drop("LB_UNME", axis=1)
food_data_2014_scaled = pd.DataFrame(scaler.fit_transform(food_data_2014_scaled))
food_data_2014_scaled["LB_UNME"] = train_unmet

test_unmet = food_data_2015["LB_UNME"].values
food_data_2015_scaled = food_data_2015.drop("LB_UNME", axis=1)
food_data_2015_scaled = pd.DataFrame(scaler.fit_transform(food_data_2015_scaled))
food_data_2015_scaled["LB_UNME"] = test_unmet

In [820]:
train_y = food_data_2014_scaled['LB_UNME'].values
train_x = food_data_2014_scaled.drop('LB_UNME', axis=1)
xtra, xte, ytra, yte = train_test_split(
    train_x, train_y, test_size=0.2, random_state=2019)

rf_params = {}
rf_params["n_estimators"] = 100
rf_params["max_depth"] = 5
rf_params["random_state"] = 2019

rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(xtra, ytra)
train_preds = rf_model.predict(xte)

test_y = food_data_2015_scaled["LB_UNME"]
test_x = food_data_2015_scaled.drop('LB_UNME', axis=1)
test_preds = rf_model.predict(test_x)

print("Current R2: {}".format(r2_score(yte, train_preds)))
print("Current RMSE: {}".format(math.sqrt(mean_squared_error(yte, train_preds))))
print("Current R2 (2015): {}".format(r2_score(test_y, test_preds)))
print("Current RMSE (2015): {}".format(math.sqrt(mean_squared_error(test_y, test_preds))))

Current R2: 0.930685333553116
Current RMSE: 11018.732290527003
Current R2 (2015): 0.8770873438216746
Current RMSE (2015): 15484.192364371012


These results are much better. Here we can see that, when scaled, we can actually get a pretty good sense of the target variable throughout, and that our model actually generalizes pretty well when predicting this unknown data. Let's see if we can improve the score further through a bit of feature engineering and/or polynomial features added to the dataset.

In [821]:
poly = PolynomialFeatures(degree = 2)

train_unmet = food_data_2014_scaled["LB_UNME"].values
food_data_2014_poly = food_data_2014_scaled.drop("LB_UNME", axis=1)
food_data_2014_poly = pd.DataFrame(poly.fit_transform(food_data_2014_poly))
food_data_2014_poly["LB_UNME"] = train_unmet

test_unmet = food_data_2015_scaled["LB_UNME"].values
food_data_2015_poly = food_data_2015_scaled.drop("LB_UNME", axis=1)
food_data_2015_poly = pd.DataFrame(poly.fit_transform(food_data_2015_poly))
food_data_2015_poly["LB_UNME"] = train_unmet

In [822]:
train_y = food_data_2014_poly['LB_UNME'].values
train_x = food_data_2014_poly.drop('LB_UNME', axis=1)
xtra, xte, ytra, yte = train_test_split(
    train_x, train_y, test_size=0.2, random_state=2019)

rf_params = {}
rf_params["n_estimators"] = 100
rf_params["max_depth"] = 5
rf_params["random_state"] = 2019

rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(xtra, ytra)
train_preds = rf_model.predict(xte)

test_y = food_data_2015_poly["LB_UNME"]
test_x = food_data_2015_poly.drop('LB_UNME', axis=1)
test_preds = rf_model.predict(test_x)

print("Current R2: {}".format(r2_score(yte, train_preds)))
print("Current RMSE: {}".format(math.sqrt(mean_squared_error(yte, train_preds))))
print("Current R2 (2015): {}".format(r2_score(test_y, test_preds)))
print("Current RMSE (2015): {}".format(math.sqrt(mean_squared_error(test_y, test_preds))))

Current R2: 0.9290810974470055
Current RMSE: 11145.513083202444
Current R2 (2015): 0.7788032604521672
Current RMSE (2015): 20245.909858644583


It doesn't look like these features will end up being helpful throughout, and that there aren't that many useful interactions to be found from polynomial features. Overall, though, given that we've dropped the majority of attributes and the above statement is true, perhaps we should look further into model building and the like to improve our results as a whole.

In [823]:
class SklearnWrapper(object):

    def __init__(self, clf, params=None):
        self.clf = clf(**params)

    def train(self, train, target, splits, mute=False):
        
        train_preds = np.zeros((len(train)))
        
        for index, (train_idx, valid_idx) in enumerate(splits):
            train_x = np.array(train)[train_idx.astype(int)]
            train_y = np.array(target)[train_idx.astype(int)]
            valid_x = np.array(train)[valid_idx.astype(int)]
            valid_y = np.array(target)[valid_idx.astype(int)]
            
            self.clf.fit(train_x, train_y)
            split_preds = self.clf.predict(valid_x)
            train_preds[valid_idx] = split_preds

        if mute == False:
            print("Global Train R2: {}".format(r2_score(target, train_preds)))
            print("Global Train RMSE: {}".format(math.sqrt(mean_squared_error(target, train_preds))))
            
        return train_preds

    def predict(self, x):
        
        test_preds = self.clf.predict(x)
        return test_preds

    def predict_test(self, x, target, mute=False):
        
        test_preds = self.clf.predict(x)
        
        if mute == False:
            print("Global Test R2: {}".format(r2_score(target, test_preds)))
            print("Global Test RMSE: {}".format(math.sqrt(mean_squared_error(target, test_preds))))
            
        return test_preds

In [824]:
n_splits = 5
seed = 2019

train_y = food_data_2014_scaled['LB_UNME'].values
train_x = food_data_2014_scaled.drop('LB_UNME', axis=1)
test_y = food_data_2015_scaled["LB_UNME"]
test_x = food_data_2015_scaled.drop('LB_UNME', axis=1)

splits = list(KFold(n_splits=n_splits, shuffle=True, random_state=seed).split(train_x, train_y))

In order to improve the accuracy of our CV, each individual model will be trained and optimized on five different splits of the training data, which again consists of the 2014 data gathered and explored above. The models will specifically predict the amount of unmet need experienced after the 2015 distribution, also as gathered and explored above.

In [825]:
rf_params = {}
rf_params['n_estimators'] = 100
rf_params['random_state'] = 2019

rf_model = SklearnWrapper(clf=RandomForestRegressor, params=rf_params)
rf_train_preds = rf_model.train(train_x, train_y, splits).reshape(-1, 1)
rf_test_preds = rf_model.predict_test(test_x, test_y)

Global Train R2: 0.9637160343403612
Global Train RMSE: 8199.83874777421
Global Test R2: 0.940738521117752
Global Test RMSE: 10751.684172598094


In [826]:
et_params = {}
et_params['n_estimators'] = 100
et_params['random_state'] = 2019

et_model = SklearnWrapper(clf=ExtraTreesRegressor, params=et_params)
et_train_preds = et_model.train(train_x, train_y, splits).reshape(-1, 1)
et_test_preds = et_model.predict_test(test_x, test_y)

Global Train R2: 0.9805915091511428
Global Train RMSE: 5997.134953779228
Global Test R2: 0.9562164313818313
Global Test RMSE: 9241.571632967785


In [827]:
lr_params = {}

lr_model = SklearnWrapper(clf=LinearRegression, params=lr_params)
lr_train_preds = lr_model.train(train_x, train_y, splits).reshape(-1, 1)
lr_test_preds = lr_model.predict_test(test_x, test_y)

Global Train R2: 0.792642078358342
Global Train RMSE: 19602.35517671075
Global Test R2: 0.7953165206701296
Global Test RMSE: 19981.663979803947


In [828]:
ri_params = {}

ri_model = SklearnWrapper(clf=Ridge, params=ri_params)
ri_train_preds = ri_model.train(train_x, train_y, splits).reshape(-1, 1)
ri_test_preds = ri_model.predict_test(test_x, test_y)

Global Train R2: 0.7926437667265711
Global Train RMSE: 19602.27537252289
Global Test R2: 0.7952903999113028
Global Test RMSE: 19982.93892289358


In [829]:
la_params = {}

la_model = SklearnWrapper(clf=Lasso, params=la_params)
la_train_preds = la_model.train(train_x, train_y, splits).reshape(-1, 1)
la_test_preds = la_model.predict_test(test_x, test_y)

Global Train R2: 0.7926415377189162
Global Train RMSE: 19602.3807310736
Global Test R2: 0.7953163881350945
Global Test RMSE: 19981.670448987774


In [830]:
en_params = {}

en_model = SklearnWrapper(clf=LinearRegression, params=en_params)
en_train_preds = en_model.train(train_x, train_y, splits).reshape(-1, 1)
en_test_preds = en_model.predict_test(test_x, test_y)

Global Train R2: 0.792642078358342
Global Train RMSE: 19602.35517671075
Global Test R2: 0.7953165206701296
Global Test RMSE: 19981.663979803947


In [831]:
gb_params = {}
gb_params['random_state'] = 2019

gb_model = SklearnWrapper(clf=GradientBoostingRegressor, params=gb_params)
gb_train_preds = gb_model.train(train_x, train_y, splits).reshape(-1, 1)
gb_test_preds = gb_model.predict_test(test_x, test_y)

Global Train R2: 0.9711114903988048
Global Train RMSE: 7316.620113500541
Global Test R2: 0.963777594270698
Global Test RMSE: 8405.797074801845


In [832]:
ab_params = {}
ab_params['n_estimators'] = 100
ab_params['random_state'] = 2019

ab_model = SklearnWrapper(clf=AdaBoostRegressor, params=ab_params)
ab_train_preds = ab_model.train(train_x, train_y, splits).reshape(-1, 1)
ab_test_preds = ab_model.predict_test(test_x, test_y)

Global Train R2: 0.8474795246360962
Global Train RMSE: 16811.7147822436
Global Test R2: 0.8033925258229204
Global Test RMSE: 19583.497998894356


In [833]:
train_second_layer = np.concatenate((
    train_x,
    rf_train_preds, 
    et_train_preds, 
    #lr_train_preds, 
    #ri_train_preds, 
    #la_train_preds, 
    #en_train_preds, 
    gb_train_preds, 
    #ab_train_preds,
), axis=1)

test_second_layer = np.concatenate((
    test_x,
    rf_test_preds[:, None], 
    et_test_preds[:, None], 
    #lr_test_preds[:, None], 
    #ri_test_preds[:, None], 
    #la_test_preds[:, None], 
    #en_test_preds[:, None], 
    gb_test_preds[:, None], 
    #ab_test_preds[:, None],
), axis=1)

et2_params = {}
et2_params['n_estimators'] = 200
et2_params['max_depth'] = 10
et2_params['random_state'] = 2019

et2_model = SklearnWrapper(clf=ExtraTreesRegressor, params=et2_params)
et2_train_preds = et2_model.train(train_second_layer, train_y, splits).reshape(-1, 1)
et2_test_preds = et2_model.predict_test(test_second_layer, test_y)

Global Train R2: 0.9814907746492356
Global Train RMSE: 5856.552756664617
Global Test R2: 0.9682178427428052
Global Test RMSE: 7873.755815742421


After finalizing and tuning the stacking ensemble, we were able to reach an R2 score of approximately 0.96821 and an RMSE score of approximately 7873.75 on the test set, quite a bit higher than any of the individual first-layer models throughout. Following this, the next step is to use this model and the information gathered to go and see if we're successfully able to minimize unmet need in the 2015 distribution.

In [834]:
sum_unmet = sum(test_y)
print(sum_unmet)

59288756.15181693


In [835]:
sum_unmet_pred = sum(et2_test_preds)
print(sum_unmet_pred)

63151744.30407871


The above stacking ensemble made use of three individual model predictions on top of the initial train/test data. The next step is to implement a pipeline to quickly generate scores/predictions, allowing us to modify the distribution as necessary in order to minimize the unmet need throughout. 

In [836]:
class EnsemblePipeline(object):
        
    def transform(self, train, test):
        
        scaler = StandardScaler()
        train_unmet = train["LB_UNME"].values
        test_unmet = test["LB_UNME"].values

        train = train.drop("LB_UNME", axis=1)
        train = pd.DataFrame(scaler.fit_transform(train))
        train["LB_UNME"] = train_unmet
        
        test = test.drop("LB_UNME", axis=1)
        test = pd.DataFrame(scaler.fit_transform(test))
        test["LB_UNME"] = test_unmet
        
        return train, test
        
    def gen_predictions(self, train, test, second_layer_params):
        
        n_splits = 5
        seed = 2019

        train_y = train['LB_UNME'].values
        train_x = train.drop('LB_UNME', axis=1)
        test_y = test["LB_UNME"]
        test_x = test.drop('LB_UNME', axis=1)

        splits = list(KFold(n_splits=n_splits, shuffle=True, random_state=seed).split(train_x, train_y))
        
        rf_model = SklearnWrapper(clf=RandomForestRegressor, params={
            'n_estimators': 100, 'random_state': 2019})
        rf_train_preds = rf_model.train(train_x, train_y, splits, mute=True).reshape(-1, 1)
        rf_test_preds = rf_model.predict_test(test_x, test_y, mute=True)

        et_model = SklearnWrapper(clf=ExtraTreesRegressor, params={
            'n_estimators': 100, 'random_state': 2019})
        et_train_preds = et_model.train(train_x, train_y, splits, mute=True).reshape(-1, 1)
        et_test_preds = et_model.predict_test(test_x, test_y, mute=True)
        
        gb_model = SklearnWrapper(clf=GradientBoostingRegressor, params={
            'random_state': 2019})
        gb_train_preds = gb_model.train(train_x, train_y, splits, mute=True).reshape(-1, 1)
        gb_test_preds = gb_model.predict_test(test_x, test_y, mute=True)

        train_second_layer = np.concatenate((
            train_x, rf_train_preds, et_train_preds, gb_train_preds), axis=1)
        test_second_layer = np.concatenate((
            test_x, rf_test_preds[:, None], et_test_preds[:, None], 
            gb_test_preds[:, None]), axis=1)

        et2_model = SklearnWrapper(clf=ExtraTreesRegressor, params=second_layer_params)
        et2_train_preds = et2_model.train(train_second_layer, train_y, splits, mute=True).reshape(-1, 1)
        et2_test_preds = et2_model.predict_test(test_second_layer, test_y, mute=True)
        
        return et2_train_preds, et2_test_preds, test_y

Given the above pipeline, all that we need to do now is modify the distribution throughout the 2015 test set and see whether or not we can reduce the overall unmet need. In order to do so, we'll specifically take the sum of the given distribution and redistribute it as necessary, specifically ensuring that we don't end up distributing more than the amount that we have to work with.

In [837]:
food_data = pd.read_csv("Capital_Area_Food_Bank_Hunger_Estimates.csv")

In [838]:
f15_columns = ['F15_FI_RATE', 'F15_LB_UNME', 'F15_DISTRIB', 'F15_PPIN']
f14_columns = ['F14_FI_RATE', 'F14_LB_UNME', 'F14_DISTRIB', 'F14_PPIN']

f14_initial = [col for col in food_data.columns if col not in f14_columns]
f14_final = [col for col in f14_initial if col not in f15_columns] + f14_columns
f15_initial = [col for col in food_data.columns if col not in f15_columns]
f15_final = [col for col in f15_initial if col not in f14_columns] + f15_columns

In [839]:
food_data_2014 = food_data[f14_final]
food_data_2015 = food_data[f15_final]

f14_rename = {'F14_FI_RATE': "FI_RATE", 'F14_LB_UNME': "LB_UNME", 'F14_DISTRIB': "DISTRIB", 'F14_PPIN': "PPIN"}
f15_rename = {'F15_FI_RATE': "FI_RATE", 'F15_LB_UNME': "LB_UNME", 'F15_DISTRIB': "DISTRIB", 'F15_PPIN': "PPIN"}
cols_to_use = ["LB_UNME", "FI_RATE", "TOTAL_POP", "PPIN", "DISTRIB"]

food_data_2014 = food_data_2014.rename(columns=f14_rename)[cols_to_use]
food_data_2015 = food_data_2015.rename(columns=f15_rename)[cols_to_use]

In [840]:
second_params = {}
second_params["n_estimators"] = 200
second_params["max_depth"] = 5
second_params["random_state"] = 2019

pipeline = EnsemblePipeline()
train, test = pipeline.transform(food_data_2014, food_data_2015)
train_preds, test_preds, test_y = pipeline.gen_predictions(train, test, second_params)
print("Calculated Unmet Need (Difference): {}".format(sum(test_preds) - sum(test_y)))

Calculated Unmet Need (Difference): 3525342.068632804


The initial difference of unmet need from the predictions comes out to be approximately 3525342.07. We'll go and set this value as the "zero" and see if we can go and improve on this result with the addition of noise and a few other strategies intended to minimize this value.

In [841]:
print(sum(food_data_2015["DISTRIB"]))

34029764.92214297


It looks like we have just over 34,000,000 pounds of food to distribute. One way that we could do such is to simply look at the population of each individual district and the percentage of individuals in the district whom are experiencing food insecurity.

In [842]:
district_population = food_data_2015["TOTAL_POP"].values
insecurity_rate = food_data_2015["FI_RATE"].values

district_percentages = [x * y for x, y in zip(district_population, insecurity_rate)]
normalized_percentages = [(x - min(district_percentages)) / (max(district_percentages) - min(district_percentages)) for x in district_percentages]
distribution_values = [sum(food_data_2015["DISTRIB"]) * x / (sum(normalized_percentages)) for x in normalized_percentages]

In [843]:
food_data_2015["DISTRIB"] = distribution_values
print(sum(food_data_2015["DISTRIB"]))

34029764.92214292


In [844]:
second_params = {}
second_params["n_estimators"] = 200
second_params["max_depth"] = 1
second_params["random_state"] = 2019
            
pipeline = EnsemblePipeline()
train, test = pipeline.transform(food_data_2014, food_data_2015)
train_preds, test_preds, test_y = pipeline.gen_predictions(train, test, second_params)
print("Calculated Unmet Need (Difference): {}".format(sum(test_preds) - sum(test_y)))

Calculated Unmet Need (Difference): 2632726.583172731


After reworking the overall distribution methodology, we were able to get a unmet need calculation of 2632726.58, down from 3525342.07 in the original simulation. Although we're not exactly able to get a complete idea of how well the new methodology will work throughout, given that the only true labels we're able to get are the ones for the original, the calculated unmet need with this distribution is expected to be approximately 892615.49 less than it would have been, going to show that Machine Learning truly can help with problems such as these.